__Author: Manu Jayadharan, University of Pittsburgh, 2020__

# Example 2: Using fluidlearn to solve an elliptic pde: 3d Poission equation.

- This is the second example in the series, intended to act as tutorial for fluidlearn package. 
- New in this example: how to use one of in-built PDE models. We illustrate this by using the _Poisson_ model from the fluidlearn.fluidmodels module. 
- We also show how to manufacture boundary conditions easily using the fluidlearn.dataprocess module, for convergence testing and debugging.

Equation to solve: $-\Delta u -f  = 0$
over domain $\Omega$ .

For demonstration purposes we take $f=-6(x_1 + x_2) - 2$ and $\Omega = [-2,4]\times [0,5]\times [-3,3]$, so we can compare the results with the actual solution $u=x_1^3 + x_2^3 + x_3^2$.

In [1]:
#Import fluidlearn package and classes
import fluidlearn
from fluidlearn import dataprocess

### Defining the domain and time interval for which the PDE needs to be solved.
This matters only for generating collocation points and if the user is feeding their own collocation points,
they can skip this step.

In [14]:
#domain range
X_1_domain = [-1, 1]
X_2_domain = [-1, 1]
X_3_domain = [-1,1]

#domain of the problem
domain_bounds = [X_1_domain, X_2_domain, X_3_domain]

### Manufacturing the boundary data
- We use the fluidlearn.dataprocess.BcIcManufact class to generate points lying on the faces of the hypercube defined by the intervals given in domain_bounds. This is equivalent to randomly selecting points from the domain boundary, $\partial \Omega$.
- We then use our knowledge of the manufactured solution to manufacture the boundary conditions corresponding to these points.
- Note that for this example, we use uniform distribution to randomly select points.

In [16]:
bc_data_size = 1000 #number of data points on boundary

#object to randomly generate points lying on the boundary
bc_generator = dataprocess.BcIcDataManufact(domain_bounds)

In [17]:
X_data = bc_generator.generate_uniform_bc_ic(bc_data_size)

In [18]:
#Note that we will have bc_data_size number of instances for each boundary
#face
X_data.shape

(6000, 3)

#### Generating the boundary condition from random boundary points using the manufactured solution $u=x_1^3 + x_2^3 + x_3^2$

In [19]:
import numpy as np
Y_data = (X_data[:,0]**3 + X_data[:,1]**3 + X_data[:,2]**2)[:,np.newaxis]


In [20]:
print(X_data[0:4,0]**3 + X_data[0:4,1]**3 + X_data[0:4,2]**2)
Y_data[0:4]

[-0.04021596 -0.26791829 -0.50201533 -0.35871264]


array([[-0.04021596],
       [-0.26791829],
       [-0.50201533],
       [-0.35871264]])

### Defining the rhs function $f=-6(x_1 + x_2) - 2$ of the PDE.

In [21]:
def rhs_function (args, time_dep=False):
        return -6*(args[0]+args[1]) -2

In [22]:
X_data[0:4,:].shape

(4, 3)

### Defining the model architecture

In [23]:
model_type = 'poisson'
space_dim = 3 #dimension of Omega
time_dependent_problem = False
n_hid_lay=3 #numberof hidden layers in the neural network
n_hid_nrn=20 #number of neurons in each hidden layer
act_func='tanh' #activation function used for hidden layers:  could be elu, relu, sigmoid
loss_list='mse' #type of error function used for cost functin, we use mean squared error.
optimizer='adam' #type of optimizer for cost function minimization
dom_bounds=domain_bounds #domain bounds where collocation points has to be generated

distribution = 'uniform' #type of distribution used for generating the pde collocation points.
number_of_collocation_points = 10000

batch_size = 32 #batch size for stochastic batch gradient type optimization
num_epochs = 10 #number of epochs used for trainng  

### Defining the fluidlearn solver 

In [24]:
#Instantiation of the fluidlearn.fluildlearn.Solver class
poisson3d_model = fluidlearn.Solver()

In [25]:
poisson3d_model(model_type=model_type,
            space_dim=space_dim,
            time_dep=time_dependent_problem,
            output_dim=1,
            n_hid_lay=n_hid_lay,
            n_hid_nrn=n_hid_lay,
            act_func=act_func,
            rhs_func=rhs_function,
            loss_list=loss_list,
            optimizer=optimizer,
            dom_bounds=dom_bounds,
            load_model=False,
            model_path=None)

### Fitting the model

In [27]:
poisson3d_model.fit(
    x=X_data,
    y=Y_data,
    colloc_points=number_of_collocation_points,
    dist=distribution,
    batch_size=batch_size,
    epochs=num_epochs,
)

Epoch 1/10
500/500 [==============================] - 4s 8ms/step - loss: 2.4524 - output_1_loss: 1.1554 - output_2_loss: 1.2971
Epoch 2/10
500/500 [==============================] - 4s 8ms/step - loss: 2.1860 - output_1_loss: 1.0135 - output_2_loss: 1.1725
Epoch 3/10
183/500 [=========>....................] - ETA: 2s - loss: 1.9933 - output_1_loss: 0.9217 - output_2_loss: 1.0716

KeyboardInterrupt: 

### Resuming Training  the model again for 50 more epochs

In [30]:
poisson3d_model.fit(
    x=X_data,
    y=Y_data,
    colloc_points=number_of_collocation_points,
    dist=distribution,
    batch_size=batch_size,
    epochs=50,
)

Epoch 1/50
500/500 [==============================] - 4s 8ms/step - loss: 671.8516 - output_1_loss: 415.1599 - output_2_loss: 256.6917
Epoch 2/50
500/500 [==============================] - 4s 8ms/step - loss: 653.8682 - output_1_loss: 402.1842 - output_2_loss: 251.6844
Epoch 3/50
500/500 [==============================] - 4s 8ms/step - loss: 634.7609 - output_1_loss: 388.9823 - output_2_loss: 245.7783
Epoch 4/50
500/500 [==============================] - 4s 8ms/step - loss: 615.6924 - output_1_loss: 378.9070 - output_2_loss: 236.7856
Epoch 5/50
500/500 [==============================] - 4s 9ms/step - loss: 596.2515 - output_1_loss: 369.2776 - output_2_loss: 226.9734
Epoch 6/50
500/500 [==============================] - 4s 8ms/step - loss: 576.6564 - output_1_loss: 360.9432 - output_2_loss: 215.7130
Epoch 7/50
500/500 [==============================] - 4s 9ms/step - loss: 552.8809 - output_1_loss: 358.6661 - output_2_loss: 194.2148
Epoch 8/50
500/500 [==============================] - 5

### Demo Using the trained model for predicton

In [ ]:
# X_test = 

In [32]:
#taking two points from the domain for time t=0.3 and t=0.76 respectively
x_test_points = [[-0.5,0.1,0.3],
                [0.66,0.6,0.76]]
#Predicting the value
y_predicted = poisson3d_model.predict(x_test_points)

In [33]:
#finding the true y value for comparing
import numpy as np
x_test_points = np.array(x_test_points)
y_true = np.sin(x_test_points[:,0:1] + x_test_points[:,1:2]) * x_test_points[:,2:3]

In [36]:
#looking at predicted and true solution side by side.

In [37]:
np.concatenate([y_predicted, y_true], axis=1)

array([[-0.1297535 , -0.1168255 ],
       [ 0.70116615,  0.72358866]])

Note that we need more training for further improving the accuracy.

### Saving the model to a specified location.

In [48]:
path_to_save_model = "saved_model/model_name"
poisson3d_model.save_model(path_to_save_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/model_name\assets


### Loading the saved model 

In [6]:
path_to_load_model = "saved_model/model_name"

In [7]:
loaded_poisson3d_model = fluidlearn.Solver()

In [8]:
loaded_poisson3d_model(space_dim=2,
    time_dep=True,
    load_model=True,
    model_path=path_to_load_model)

### Predicting using loaded model

In [9]:
y_predicted = loaded_poisson3d_model.predict(X_data)

In [10]:
y_predicted

array([[-0.10157388],
       [-0.4190994 ],
       [-0.7965628 ],
       ...,
       [-0.10375804],
       [ 0.05802408],
       [-0.00470909]], dtype=float32)